In [1]:
import sys
sys.path.append('./src')
import syllo_gen
import random
import json
from tqdm.autonotebook import tqdm
import openai
openai.api_key = open('key.txt').read().strip()

%load_ext autoreload
%autoreload 2

C:\Users\cheny\AppData\Local\Temp\ipykernel_25688\2018132128.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
sum([len(x) for x in syllo_gen.ADJ_TEMPLATE.values()])

66

In [15]:
q,v = syllo_gen.get_syllo(2)
# q = syllo_gen.negate_quesion(q)
question = syllo_gen.question2fol(q, v)
for line in question['story'] + [question['conclusion']]:
    print(line)
print()
v1 = syllo_gen.random_assign_adjs(v)

question = syllo_gen.question2template(q, v1, rand=True, noun=False)
for line in question['story'] + [question['conclusion']]:
    print(line)
print()
v2 = syllo_gen.random_assign_nouns(v)

question = syllo_gen.question2template(q, v2, rand=True, noun=True)
for line in question['story'] + [question['conclusion']]:
    print(line)


∃x (B(x) ∧ C(x))
∀x (B(x) → A(x))
∀x (A(x) → ¬D(x))
∃x (C(x) ∧ ¬D(x))

Thrilling and intellectual person exists.
Everyone who is thrilling is soulless.
For all people, if he is soulless then he is not glamorous.
Someone is intellectual and not glamorous.

A subset of fire is owner.
All fire is carton of milk.
If something is carton of milk, then it is not reputation.
Sometimes owner is not reputation.


# ADJ Template

In [4]:
q,v = syllo_gen.get_syllo(1)
# q = syllo_gen.negate_quesion(q)
v = syllo_gen.random_assign_adjs(v)
print(syllo_gen.question2fol(q, v))
print(syllo_gen.question2template(q, v, rand=True, noun=False))

{'story': ['∀x (celestial(x) → ¬apprehensive(x))', '∃x (celestial(x) ∧ inconsiderate(x))'], 'conclusion': '∃x (inconsiderate(x) ∧ ¬apprehensive(x))'}
{'story': ['It is impossible for a celestial person to be apprehensive.', 'There is someone who is celestial and inconsiderate.'], 'conclusion': 'Some people are inconsiderate and not apprehensive.'}


# NOUN Template

In [2]:
q,v = syllo_gen.get_syllo(1)
# q = syllo_gen.negate_quesion(q)
v = syllo_gen.random_assign_nouns(v)  # change to correct type
print(syllo_gen.question2fol(q, v))
print(syllo_gen.question2template(q, v, rand=True, noun=True)) # change to correct type


{'story': ['∀x (gene(x) → ¬line(x))', '∀x (choice(x) → gene(x))'], 'conclusion': '∃x (choice(x) ∧ ¬line(x))'}
{'story': ['Gene is not line.', 'There is no choice that is not gene.'], 'conclusion': 'Not all choice is line.'}


In [16]:
all_possilbe = set()
# prompt = 'Answer questions about syllogisms, ignoring semantics.\n\n'
prompt = 'Answer questions about syllogisms.\n\n'
# prompt = ''
rand = False
noun = False
for j in range(100):
    if len(all_possilbe) == 24:
        break
    real = random.choice([True, False])
    q, v = syllo_gen.get_syllo(1)

    qs = syllo_gen.question_to_string(q)
    if qs in all_possilbe:
        continue
    all_possilbe.add(qs)

    if not real:
        q = syllo_gen.negate_quesion(q)

    # if random.random() < 0.5:
    #     v = syllo_gen.assign_greek_letters(v)

        
    if noun:
        v = syllo_gen.random_assign_nouns(v)
    else:
        v = syllo_gen.random_assign_adjs(v)

    
    q = syllo_gen.question2template(q, v, rand=rand, noun=noun)
    # q = syllo_gen.question2fol(q, v)

    prompt += f'Story: {" ".join(q["story"])} Question: {q["conclusion"]}\n'
    prompt += f'Answer: {real}\n\n'

# for j in range(100):
#     if len(all_possilbe) == 30:
#         break
#     real = random.choice([True, False])
#     q, v = syllo_gen.get_syllo(2)
#     if not real:
#         q = syllo_gen.negate_quesion(q)

#     v = syllo_gen.random_assign_nouns(v)
#     if tuple(q['story']) not in all_possilbe:
#         all_possilbe.add((tuple(q['story'])))
#         q = syllo_gen.question2template(q, v, rand=rand)
#         prompt += f'Story: {" ".join(q["story"])} Question: {q["conclusion"]}\n'
#         prompt += f'Answer: {real}\n\n'
print(prompt)

Answer questions about syllogisms.

Story: No wicked person is flashy. Some divine person is wicked. Question: Every divine person is flashy.
Answer: False

Story: No pleasant person is vulnerable. Every elfin person is vulnerable. Question: Some elfin person is not pleasant.
Answer: True

Story: Some drab person is squat. Every drab person is reclusive. Question: Some reclusive person is squat.
Answer: True

Story: Every faithful person is tough. No tough person is good. Question: Some good person is not faithful.
Answer: True

Story: No creepy person is indiscreet. Every self-confident person is creepy. Question: Some self-confident person is indiscreet.
Answer: False

Story: Every clever person is quick. Every chic person is clever. Question: Some chic person is quick.
Answer: True

Story: Every mellow person is attentive. Some mellow person is musical. Question: Some musical person is attentive.
Answer: True

Story: No self-disciplined person is inexperienced. Some inexperienced pe

In [ ]:
corret = 0
total= 1000
evaled = 0
all_possilbe = set()

for i in tqdm(range(total)):

    # if len(all_possilbe) == 48:
    #     break

    real = random.choice([True, False])
    q, v = syllo_gen.get_syllo(1)
    if not real:
        q = syllo_gen.negate_quesion(q)

    # qs = syllo_gen.question_to_string(q)
    # if qs not in all_possilbe:
    #     all_possilbe.add(qs)
    # else:
    #     continue

    # if random.random() < 1:
    #     v = syllo_gen.assign_greek_letters(v)

    if noun:
        v = syllo_gen.random_assign_nouns(v)
    else:
        v = syllo_gen.random_assign_adjs(v)

    q = syllo_gen.question2template(q, v, rand=rand, noun=noun)
    # q = syllo_gen.question2fol(q, v)

    tmp_prompt = prompt + f'Story: {" ".join(q["story"])} Question: {q["conclusion"]}'

    tmp_prompt += "\nAnswer:" 

    # print(tmp_prompt)
    # break


    response = openai.Completion.create(
    model="text-davinci-003",
    prompt=tmp_prompt,
    temperature=0,
    max_tokens=10,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["\n"]
    )
    text = response['choices'][0]['text'].strip()
    if text not in ['True', 'False']:
        print('Error')
        print(text)
        continue
    if text == 'True' and real:
        corret += 1
    elif text == 'False' and not real:
        corret += 1
    else:
        print(f'Story: {" ".join(q["story"])} Question: {q["conclusion"]}')
        print(text)


print(corret/total)